In [ ]:
import numpy as np
import torch 
import pyro
import re
import pandas as pd

import importlib
import time
import pickle

from granch_utils import init_model_tensor, main_sim_tensor, lesioned_sim, compute_prob_tensor, init_params_tensor, num_stab_help, proxy_sim
#importlib.reload(granch_utils)
importlib.reload(num_stab_help)
importlib.reload(init_model_tensor)
importlib.reload(main_sim_tensor)
importlib.reload(lesioned_sim)

# Main TEST

In [ ]:
importlib.reload(main_sim_tensor)
importlib.reload(lesioned_sim)
importlib.reload(init_params_tensor)
importlib.reload(compute_prob_tensor)
importlib.reload(proxy_sim)


device = 'cuda' if torch.cuda.is_available() else 'cpu'

BATCH_INFO = {
    "jitter_n": 1, 
    "total_batch_n": 1, 
    "jitter_mode": "sampling"
}

GRID_INFO = {
    "grid_mu_start": -4, "grid_mu_end": 4, "grid_mu_step": 5, 
    "grid_sigma_start": 0.001, "grid_sigma_end": 1.8, "grid_sigma_step": 5, 
    "grid_y_start": -4, "grid_y_end": 4, "grid_y_step": 5, 
    "grid_epsilon_start": 0.000001, "grid_epsilon_end": 1, "grid_epsilon_step": 5, 
    "hypothetical_obs_grid_n": 10
}


BATCH_GRID_INFO = num_stab_help.get_batch_grid(BATCH_INFO, GRID_INFO)

PRIOR_INFO = {
    "mu_prior": 1,  
    "V_prior": 1, 
    "alpha_prior": 3, 
    "beta_prior": 1, 
    "epsilon": 0.0001, "mu_epsilon":0.0001, "sd_epsilon": 0.0001, 
    "hypothetical_obs_grid_n": 2, 
    "world_EIGs": 3, "max_observation": 500
}

tensor_stimuli = num_stab_help.sample_spore_experiment(pair_each_stim = 1, n_feature=3)
print(tensor_stimuli)


tensor_model =  init_model_tensor.granch_model(PRIOR_INFO['max_observation'], tensor_stimuli[0])

params = init_params_tensor.granch_params(
                grid_mu =  BATCH_GRID_INFO["grid_mus"][0].to(device),
                grid_sigma = BATCH_GRID_INFO["grid_sigmas"][0].to(device),
                grid_y = BATCH_GRID_INFO["grid_ys"][0].to(device),
                grid_epsilon = BATCH_GRID_INFO["grid_epsilons"][0].to(device),
                hypothetical_obs_grid_n = PRIOR_INFO["hypothetical_obs_grid_n"], 
                mu_prior = PRIOR_INFO["mu_prior"],
                V_prior = PRIOR_INFO["V_prior"], 
                alpha_prior = PRIOR_INFO["alpha_prior"], 
                beta_prior = PRIOR_INFO["beta_prior"],
                epsilon  = PRIOR_INFO["epsilon"], 
                mu_epsilon = PRIOR_INFO["mu_epsilon"], 
                sd_epsilon = PRIOR_INFO["sd_epsilon"], 
                world_EIGs = PRIOR_INFO["world_EIGs"],
                max_observation = PRIOR_INFO["max_observation"], 
                forced_exposure_max= np.nan, 
                linking_hypothesis = "surprisal")
        
            # add the various different cached bits
params.add_meshed_grid()
params.add_lp_mu_sigma()
params.add_y_given_mu_sigma()
params.add_lp_epsilon()
params.add_priors()

if params.linking_hypothesis == "EIG": 
    res = main_sim_tensor.granch_main_simulation(params, tensor_model, tensor_stimuli[0])
else: 
    res = proxy_sim.granch_proxy_sim(params, tensor_model, tensor_stimuli[0])
#res = lesioned_sim.granch_no_learning_simulation(params, tensor_model, tensor_stimuli[0])
#res = lesioned_sim.granch_no_noise_simulation(params, tensor_model, tensor_stimuli[0])



pd.set_option('display.max_rows', None)

print(res.behavior)
#main_sim_tensor.granch_main_simulation(PRIOR_INFO, tensor_model, tensor_stimuli)



# Specific Grid Test

In [52]:
importlib.reload(main_sim_tensor)
importlib.reload(lesioned_sim)
importlib.reload(init_params_tensor)
importlib.reload(compute_prob_tensor)
importlib.reload(proxy_sim)
from torch.distributions import Normal, uniform


# test stimuli
test_stimuli = init_model_tensor.granch_stimuli(3, "BBBBBD")
test_stimuli.add_toy_example(0.3, 0.7)

# test grid  
grid_mu_distribution = uniform.Uniform(.2, .8)
grid_sigma_distribution = uniform.Uniform(.2, .8)
grid_y_distribution = uniform.Uniform(.2, .8)
grid_epsilon_distribution = uniform.Uniform(.2, .8)

grid_mu = grid_mu_distribution.sample([5, ])
grid_sigma = grid_sigma_distribution.sample([5, ])
grid_y = grid_y_distribution.sample([5, ])
grid_epsilon = grid_epsilon_distribution.sample([5, ])

#grid_mu = torch.linspace(start = .2, end  = .8, steps = 5)
#grid_sigma = torch.linspace(start = .2, end  = .8, steps = 5)
#grid_y = torch.linspace(start = .2, end  = .8, steps = 5)
#grid_epsilon = torch.linspace(start = .2, end  = .8, steps = 5)

# test prior
PRIOR_INFO = {
    "mu_prior": 1,  
    "V_prior": 1, 
    "alpha_prior": 3, 
    "beta_prior": 1, 
    "epsilon": 0.0001, "mu_epsilon":0.0001, "sd_epsilon": 0.0001, 
    "hypothetical_obs_grid_n": 2, 
    "world_EIGs": 3, "max_observation": 500
}


tensor_model =  init_model_tensor.granch_model(PRIOR_INFO['max_observation'], test_stimuli)

params = init_params_tensor.granch_params(
                grid_mu =  grid_mu,
                grid_sigma = grid_sigma,
                grid_y = grid_y, 
                grid_epsilon = grid_epsilon,
                hypothetical_obs_grid_n = 5, 
                mu_prior = PRIOR_INFO["mu_prior"],
                V_prior = PRIOR_INFO["V_prior"], 
                alpha_prior = PRIOR_INFO["alpha_prior"], 
                beta_prior = PRIOR_INFO["beta_prior"],
                epsilon  = PRIOR_INFO["epsilon"], 
                mu_epsilon = PRIOR_INFO["mu_epsilon"], 
                sd_epsilon = PRIOR_INFO["sd_epsilon"], 
                world_EIGs = PRIOR_INFO["world_EIGs"],
                max_observation = PRIOR_INFO["max_observation"], 
                forced_exposure_max= np.nan, 
                linking_hypothesis = "surprisal")
        
            # add the various different cached bits
params.add_meshed_grid()
params.add_lp_mu_sigma()
params.add_y_given_mu_sigma()
params.add_lp_epsilon()
params.add_priors()

if params.linking_hypothesis == "EIG": 
    res = main_sim_tensor.granch_main_simulation(params, tensor_model, test_stimuli)
else: 
    res = proxy_sim.granch_proxy_sim(params, tensor_model, test_stimuli)

res

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.29983070492744446' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.2999591827392578' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.2999488413333893' has dtype incompatible with int64, please explicitly cast to a compatible dt

,t,kl1,kl2
0,0,0.103386,-0.064521
1,1,0.000229,0.00023
2,2,0.00008,0.00008
3,3,0.000032,0.000032
4,4,0.000017,0.000017
5,5,0.000011,0.000011
6,6,0.000008,0.000008
7,7,0.000007,0.000007
8,8,0.000006,0.000006
9,9,0.000005,0.000005


In [50]:
res

# Visualizing random grid KL

In [53]:
importlib.reload(main_sim_tensor)
importlib.reload(lesioned_sim)
importlib.reload(init_params_tensor)
importlib.reload(compute_prob_tensor)
importlib.reload(proxy_sim)


def run_random_grid_and_visualize_kl():

    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    BATCH_INFO = {
        "jitter_n": 1, 
        "total_batch_n": 1, 
        "jitter_mode": "sampling"
    }

    GRID_INFO = {
       "grid_mu_start": -4, "grid_mu_end": 4, "grid_mu_step": 20, 
       "grid_sigma_start": 0.001, "grid_sigma_end": 1.8, "grid_sigma_step": 20, 
       "grid_y_start": -4, "grid_y_end": 4, "grid_y_step": 20, 
       "grid_epsilon_start": 0.000001, "grid_epsilon_end": 1, "grid_epsilon_step": 20, 
       "hypothetical_obs_grid_n": 10
    }


#    GRID_INFO = {
#        "grid_mu_start": .2, "grid_mu_end": .8, "grid_mu_step": 5, 
#        "grid_sigma_start": .2, "grid_sigma_end": .8, "grid_sigma_step": 5, 
#        "grid_y_start":.2, "grid_y_end": 8, "grid_y_step": 5, 
#        "grid_epsilon_start": .2, "grid_epsilon_end": .8, "grid_epsilon_step": 5, 
#        "hypothetical_obs_grid_n": 10
#    }



    BATCH_GRID_INFO = num_stab_help.get_batch_grid(BATCH_INFO, GRID_INFO)

    PRIOR_INFO = {
        "mu_prior": 1,  
        "V_prior": 1, 
        "alpha_prior": 3, 
        "beta_prior": 1, 
        "epsilon": 0.0001, "mu_epsilon":0.0001, "sd_epsilon": 0.0001, 
        "hypothetical_obs_grid_n": 2, 
        "world_EIGs": 3, "max_observation": 500
    }


    test_stimuli = init_model_tensor.granch_stimuli(3, "BBBBBD")
    test_stimuli.add_toy_example(0.3, 0.7)



    tensor_model =  init_model_tensor.granch_model(PRIOR_INFO['max_observation'], test_stimuli)

    params = init_params_tensor.granch_params(
                    grid_mu =  BATCH_GRID_INFO["grid_mus"][0].to(device),
                    grid_sigma = BATCH_GRID_INFO["grid_sigmas"][0].to(device),
                    grid_y = BATCH_GRID_INFO["grid_ys"][0].to(device),
                    grid_epsilon = BATCH_GRID_INFO["grid_epsilons"][0].to(device),
                    hypothetical_obs_grid_n = PRIOR_INFO["hypothetical_obs_grid_n"], 
                    mu_prior = PRIOR_INFO["mu_prior"],
                    V_prior = PRIOR_INFO["V_prior"], 
                    alpha_prior = PRIOR_INFO["alpha_prior"], 
                    beta_prior = PRIOR_INFO["beta_prior"],
                    epsilon  = PRIOR_INFO["epsilon"], 
                    mu_epsilon = PRIOR_INFO["mu_epsilon"], 
                    sd_epsilon = PRIOR_INFO["sd_epsilon"], 
                    world_EIGs = PRIOR_INFO["world_EIGs"],
                    max_observation = PRIOR_INFO["max_observation"], 
                    forced_exposure_max= np.nan, 
                    linking_hypothesis = "surprisal")
            
                # add the various different cached bits
    params.add_meshed_grid()
    params.add_lp_mu_sigma()
    params.add_y_given_mu_sigma()
    params.add_lp_epsilon()
    params.add_priors()

    if params.linking_hypothesis == "EIG": 
        res = main_sim_tensor.granch_main_simulation(params, tensor_model, test_stimuli)
    else: 
        res = proxy_sim.granch_proxy_sim(params, tensor_model, test_stimuli)

    return res


#res = lesioned_sim.granch_no_learning_simulation(params, tensor_model, tensor_stimuli[0])
#res = lesioned_sim.granch_no_noise_simulation(params, tensor_model, tensor_stimuli[0])



# try it out 5 eims
df_list = []
for i in range(500): 
    print(i)
    res = run_random_grid_and_visualize_kl()
    res["id"] = i
    df_list.append(res)
main_res = pd.concat(df_list)
main_res.to_csv("visualize_kl_turbo_denser.csv")
#main_res.dropna()
#print(res.behavior)
#main_sim_tensor.granch_main_simulation(PRIOR_INFO, tensor_model, tensor_stimuli)



0


/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.29991069436073303' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.29985639452934265' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.30012476444244385' has dtype incompatible with int64, please explicitly cast to a compatible 

1


/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.2998548746109009' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.29997682571411133' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.2999725639820099' has dtype incompatible with int64, please explicitly cast to a compatible dt

2
3
4
5
6


...

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.30003806948661804' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.3000926077365875' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.29995858669281006' has dtype incompatible with int64, please explicitly cast to a compatible d

497
498


/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.29997915029525757' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.3000301420688629' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.30004599690437317' has dtype incompatible with int64, please explicitly cast to a compatible d

499


/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.30002638697624207' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.29990679025650024' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.30004966259002686' has dtype incompatible with int64, please explicitly cast to a compatible 

In [ ]:
import plotly.subplots as sp
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px

df = main_res

# Get unique categories
id = df['id'].unique()
# Create a subplot grid
fig = sp.make_subplots(rows=len(id), cols=1, subplot_titles=["1", "2", "3", "4", "5"])

# Populate each subplot
for i, idx in enumerate(id):
    subset = df[df['id'] == idx]
    print(subset)
    single_fig = px.line(df, x='t', y=['kl1', 'kl2'], labels={'kl1': 'kl1', 'kl2': 'kl2'}, markers=True)
    single_fig.update_layout(
    xaxis_title='t',
    yaxis_title='Value',
    legend_title='Legend',
    )


    for trace in single_fig.data:
        fig.add_trace(trace, row=i + 1, col=1)

# Update subplot layout (optional)
fig.update_layout(
    title='Faceted Line Plots',
    showlegend=True,
    height=600,
    width=400,
)

# Show the plot
fig.show()


In [ ]:
import pandas as pd
import plotly.express as px



df = res.iloc[:50]

# Create a line plot using Plotly Express
fig = px.line(df, x='t', y=['kl1', 'kl2'], labels={'kl1': 'kl1', 'kl2': 'kl2'}, markers=True)

# Customize the layout (optional)
fig.update_layout(
    title='Line Plot with Plotly',
    xaxis_title='t',
    yaxis_title='Value',
    legend_title='Legend',
)

# Show the plot
fig.show()


# Toy Example Test 

In [ ]:
importlib.reload(init_model_tensor)
importlib.reload(lesioned_sim)
importlib.reload(init_params_tensor)
importlib.reload(compute_prob_tensor)
importlib.reload(main_sim_tensor)
importlib.reload(proxy_sim)

test_stimuli = init_model_tensor.granch_stimuli(3, "BBBBBD")
test_stimuli.add_toy_example(0.3, 0.7)

tensor_model =  init_model_tensor.granch_model(PRIOR_INFO['max_observation'], tensor_stimuli[0])

params = init_params_tensor.granch_params(
                grid_mu =  BATCH_GRID_INFO["grid_mus"][0].to(device),
                grid_sigma = BATCH_GRID_INFO["grid_sigmas"][0].to(device),
                grid_y = BATCH_GRID_INFO["grid_ys"][0].to(device),
                grid_epsilon = BATCH_GRID_INFO["grid_epsilons"][0].to(device),
                hypothetical_obs_grid_n = PRIOR_INFO["hypothetical_obs_grid_n"], 
                mu_prior = PRIOR_INFO["mu_prior"],
                V_prior = PRIOR_INFO["V_prior"], 
                alpha_prior = PRIOR_INFO["alpha_prior"], 
                beta_prior = PRIOR_INFO["beta_prior"],
                epsilon  = PRIOR_INFO["epsilon"], 
                mu_epsilon = PRIOR_INFO["mu_epsilon"], 
                sd_epsilon = PRIOR_INFO["sd_epsilon"], 
                world_EIGs = 3,
                #world_EIGs = PRIOR_INFO["world_EIGs"],
                #max_observation = PRIOR_INFO["max_observation"], 
                max_observation = 500,
                forced_exposure_max= np.nan, 
                linking_hypothesis= "surprisal")
        
            # add the various different cached bits
params.add_meshed_grid()
params.add_lp_mu_sigma()
params.add_y_given_mu_sigma()
params.add_lp_epsilon()
params.add_priors()

pd.set_option('display.max_rows', None)

#res = main_sim_tensor.granch_main_simulation(params, tensor_model, test_stimuli)
res = proxy_sim.granch_proxy_sim(params, tensor_model, test_stimuli)
#res.behavior

res.behavior

In [ ]:
res.behavior